In [6]:
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Определение языка

Необходимо решить задачу определения языка на коллекция из 383,108 текстов на 26 языках.

### Подготовка данных

Для начала посмотрим на [данные](https://www.dropbox.com/s/fkr16tbrh90bq78/TLI.zip?dl=0). Первая колонка здесь – метка класса (язык), вторая – текст, а разделены они табуляцией.

In [2]:
import pandas as pd

data = pd.read_table('./data/data-lang-id.txt', sep='\t')
data.head()

In [ ]:
data.groupby('lang').count()

In [3]:
# что-то не так...
data.count()

lang    383111
text    383108
dtype: int64

In [4]:
# выкинем строки, в которых есть пустые значения
# и переназначим индексы

data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data.count()

lang    383108
text    383108
dtype: int64

Теперь нужно разбить данные на тренировочную и тестовую выборку. Кажется, они уже перемешаны, но на всякий случай перемешаем их перед разбиением еще раз.

In [5]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

data = shuffle(data)

train, test = train_test_split(data, test_size=0.2)
train.tail()

,lang,text
132941,fr,Le plaisir qu’il y prenait se reflétait sur se...
31184,ru,Алесь привязал коня к забору и медленно пошел ...
255297,en,"Ranged around the building in ring fashion, th..."
198820,pl,"Myślałem o Claire, o wieczorach, które u niej ..."
152289,ru,"Кроме нас двоих, у нее нет никого на земле. Ск..."


## Векторизация

Для работы необходимо векторизовать тексты. Это можно по-разному. Простейшие способы — `CountVectorizer` (мешок слов) и `TfidfVectorizer`.

### Мешок слов

Это значит, что каждое слово или каждая N-грамма задает свою координату в векторном пространстве и никаких дополнительных признаков – например, порядок слов – не использует. 

**N-граммы** — это сочетания из N элементов (слов, символов), идущих друг за другом. Одиночные элементы называются униграммами, сочетания из двух элементов -- биграммами, из трёх — триграммами, а дальше 4-граммы, 5-граммы и т.д. 

### TF-IDF

Ещё один способ работы с текстовыми данными — [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) *(Term Frequency – Inverse Document Frequency)*. Рассмотрим коллекцию текстов $D$.  Для каждого уникального слова $t$ из документа $d \in D$ вычислим следующие величины:

**1. Term Frequency** – количество вхождений слова в отношении к общему числу слов в тексте:
$$\text{tf}(t, d) = \frac{n_{td}}{\sum_{t \in d} n_{td}},$$
где $n_{td}$ — количество вхождений слова $t$ в текст $d$.


**2. Inverse Document Frequency**
$$\text{idf}(t, D) = \log \frac{\left| D \right|}{\left| \{d\in D: t \in d\} \right|},$$
где $\left| \{d\in D: t \in d\} \right|$ – количество текстов в коллекции, содержащих слово $t$.

Тогда для каждой пары (слово, текст) $(t, d)$ вычислим величину:
$$\text{tf-idf}(t,d, D) = \text{tf}(t, d)\cdot \text{idf}(t, D).$$

Отметим, что значение $\text{tf}(t, d)$ корректируется для часто встречающихся общеупотребимых слов при помощи значения $\text{idf}(t, D).$

Признаковым описанием одного объекта $d \in D$ будет вектор $\bigg(\text{tf-idf}(t,d, D)\bigg)_{t\in V}$, где $V$ – словарь всех слов, встречающихся в коллекции $D$.

In [4]:
vectorizer = CountVectorizer(ngram_range=(1,1))

# YOUR CODE HERE

Смотрим показания классификатора на тестовом множестве. 

In [8]:
# YOUR CODE HERE

Оцениваем качество.

In [3]:
# YOUR CODE HERE

Визуализируем результаты.

In [5]:
# YOUR CODE HERE

## Важность признаков

Веса признаков в линейной модели в случае, если признаки отмасштабированы, характеризуют степень их влияния на значение целевой переменной. В задаче классификации текстов, кроме того, признаки являются хорошо интерпретируемыми, поскольку каждый из них соответствует конкретному слову. Изучим влияние конкретных слов на значение целевой переменной.

In [6]:
f_weights = zip(vectorizer.get_feature_names(), classifier.coef_[0])
f_weights = sorted(f_weights, key=lambda i: i[1])
for i in range(1,30):
    print('%s, %.2f' % f_weights[-i])
    
print('...')
for i in reversed(range(1,10)):
    print('%s, %.2f' % f_weights[i])

### Задание

1. Векторизовать тексты с помощью `TfidfVectorizer` и проверить, влияет ли это на качество результатов классификации.
2. Скачать датасет [20newsgroups](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) и проделать то же самое с ними. В данном случае предсказывать мы будем тему новости, и, поскольку эта задача сложнее, можно сравнить качество при использовании лемматизации и без, при удалении стоп-слов и без, при разных способах векторизации, а также сравнить разные классификаторы.